In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
from scipy import stats

Upload base de dados

In [6]:
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/acoes.csv')
dataset.head()

,Date,ABEV,ODPV,CCR,PETBR,BB,BOVA
0,2015-01-02,16.020000,9.37,14.72,9.36,22.650000,47.259998
1,2015-01-05,15.730000,9.62,14.30,8.56,22.180000,46.320000
2,2015-01-06,16.330000,9.55,14.51,8.28,22.490000,46.580002
3,2015-01-07,16.559999,9.69,15.60,8.67,23.480000,48.150002
4,2015-01-08,16.540001,9.66,15.78,9.23,23.559999,48.509998


Simulação de Monte Carlo

In [4]:
def monte_carlo_previsao(dataset, ativo, dias_a_frente, simulacoes):
  dataset = dataset.copy()
  dataset = pd.DataFrame(dataset[ativo])
  
  dataset_normalizado = dataset.copy()
  for i in dataset:
    dataset_normalizado[i] = dataset[i] / dataset[i][0]
  
  dataset_taxa_retorno = np.log(1 + dataset_normalizado.pct_change())
  dataset_taxa_retorno.fillna(0, inplace=True)
  
  media = dataset_taxa_retorno.mean()
  variancia = dataset_taxa_retorno.var()

  drift = media - (0.5 * variancia)
  desvio_padrao = dataset_taxa_retorno.std()
  Z = stats.norm.ppf(np.random.rand(dias_a_frente, simulacoes))
  retornos_diarios = np.exp(drift.values + desvio_padrao.values * Z)

  previsoes = np.zeros_like(retornos_diarios)
  previsoes[0] = dataset.iloc[-1]

  for dia in range(1, dias_a_frente):
    previsoes[dia] = previsoes[dia - 1] * retornos_diarios[dia]

  figura = px.line(title = 'Previsões do preço das ações - ' + ativo)
  for i in range(len(previsoes.T)):
    figura.add_scatter(y = previsoes.T[i], name = i)
  figura.show()  

  return previsoes.T

Simulação com a ação ABEV3 com 50 dias a frente e 10 simulações

In [7]:
sim = monte_carlo_previsao(pd.read_csv('/content/drive/My Drive/Colab Notebooks/acoes.csv'), 'ABEV',50,10)

Simulação com as demais ações da base de dados

In [8]:
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/acoes.csv')
for ativo in dataset.columns[1:]:
  #print(ativo)
  monte_carlo_previsao(dataset, ativo, 100, 20)